# Insight Data Science Consulting Project: 80,000 hours - Chapter 1

Note: this is a part of a consulting project with [80,000 hours](https://80000hours.org/).

## Stage 1: Ask a question

My objective is to rank skills (and possibly knowledge, tools & tech) based on how valuable they are. The skills are listed by US Department of Labor [here](https://www.onetonline.org/find/descriptor/browse/Skills/2.B.1/).

There is no performance measure for this rank yet since it is subjective. Yet in the future, one can create a poll to rate pairwise. 

## Stage 2: Set the environment up and get data

First, set up a directory for data and link it to this workplace. Download data into your choice of directory. All data information is in the excel file ```DataDescription.xlsx```

In [1]:
#Set up the environment
import pandas as pd                        #Pandas
import numpy as np                         #Numpy
import pycurl                              #For saving file from url
import os                                  #For checking if a file exists
from pandas.parser import CParserError     #For checking if a file contains a set of values
import numbers
import decimal

# Set up data directory
DataDir = "C:/Users/Admin/Desktop/Insight/data/"

### Stage 2.1: List of occupations according to O*NET database

In [3]:
#First read the list of occupation 
filename = "All_Career_Clusters.csv"
df = pd.pandas.read_csv(DataDir+filename)

In [5]:
df.head()

,Career Cluster,Career Pathway,Code,Occupation
0,"Agriculture, Food and Natural Resources",Agribusiness Systems,25-1041.00,"Agricultural Sciences Teachers, Postsecondary"
1,"Agriculture, Food and Natural Resources",Agribusiness Systems,27-4011.00,Audio and Video Equipment Technicians
2,"Agriculture, Food and Natural Resources",Agribusiness Systems,13-1021.00,"Buyers and Purchasing Agents, Farm Products"
3,"Agriculture, Food and Natural Resources",Agribusiness Systems,25-9021.00,Farm and Home Management Advisors
4,"Agriculture, Food and Natural Resources",Agribusiness Systems,13-1074.00,Farm Labor Contractors


In [6]:
len(df)

1847

We will use the column "Code" as a way to extract information about skills and Job zone (and knowledges, tools and technology) for each occupation. Note that this code is a decimal system. We will see later than there is one without decimals.

In [7]:
CodeList = list(df.Code)

### Stage 2.2: Extract individual files from O*NET website

In [33]:
#Tools and technology
head = "https://www.onetonline.org/link/table/details/tt/"
body = "/Tools_Technology_"
tail = ".csv?fmt=csv&s=s"

for i in range(0,len(CodeList)):
    name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
    url = head + CodeList[i] + body + name + tail
    # As long as the file is opened in binary mode, both Python 2 and Python 3
    # can write response body to it without decoding.
    with open(DataDir + '/Tools_Technology/Tools_Technology_' + name +'.csv', 'wb') as f:
        c = pycurl.Curl()
        c.setopt(c.URL, url)
        c.setopt(c.WRITEDATA, f)
        c.perform()
        c.close()

In [37]:
# Knowledge
head = "https://www.onetonline.org/link/table/details/kn/"
body = "/Knowledge_"
tail = ".csv?fmt=csv&s=IM&t=-10"

for i in range(0,len(CodeList)):
    name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
    url = head + CodeList[i] + body + name + tail
    # As long as the file is opened in binary mode, both Python 2 and Python 3
    # can write response body to it without decoding.
    with open(DataDir + '/Knowledge/Knowledge_' + name +'.csv', 'wb') as f:
        c = pycurl.Curl()
        c.setopt(c.URL, url)
        c.setopt(c.WRITEDATA, f)
        c.perform()
        c.close()

In [38]:
# Skills
head = "https://www.onetonline.org/link/table/details/sk/"
body = "/Skills_"
tail = ".csv?fmt=csv&s=IM&t=-10"

for i in range(0,len(CodeList)):
    name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
    url = head + CodeList[i] + body + name + tail
    # As long as the file is opened in binary mode, both Python 2 and Python 3
    # can write response body to it without decoding.
    with open(DataDir + '/Skills/Skills_' + name +'.csv', 'wb') as f:
        c = pycurl.Curl()
        c.setopt(c.URL, url)
        c.setopt(c.WRITEDATA, f)
        c.perform()
        c.close()

In [39]:
# Job zone
head = "https://www.onetonline.org/link/table/details/jz/"
body = "/Job_Zone_"
tail = ".csv?fmt=csv"

for i in range(0,len(CodeList)):
    name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
    url = head + CodeList[i] + body + name + tail
    # As long as the file is opened in binary mode, both Python 2 and Python 3
    # can write response body to it without decoding.
    with open(DataDir + '/Job_Zone/Job_Zone_' + name +'.csv', 'wb') as f:
        c = pycurl.Curl()
        c.setopt(c.URL, url)
        c.setopt(c.WRITEDATA, f)
        c.perform()
        c.close()

We found that there are only 1,085 files in each folder. This is because some occupations has no values yet. Let's get a new CodeList. We can check just one folder.

In [63]:
TrueCodeList = []
for i in range(0,len(CodeList)):
    name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
    fname = DataDir + '/Skills/Skills_' + name +'.csv'
    if os.path.isfile(fname):
        TrueCodeList = TrueCodeList + [CodeList[i]]

In [66]:
len(CodeList), len(TrueCodeList)

(1847, 1847)

In [70]:
CodeList = TrueCodeList

Not sure why the code list still has a same length. It is fine as long as it has no error messages later.

### Stage 2.3: Construct tables of features (Skills)

First, let's look at **Skills**. It turns out the list is always fixed. We can turn it into a data table easily.

In [30]:
#Get the list of skills from the first file
i=0
name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
dsmall = pd.read_csv(DataDir + '/Skills/Skills_' + name +'.csv')
dsmall = dsmall.sort_values(by=['Skill'])

In [31]:
dSkill = pd.DataFrame(columns = ['SOC code']+list(dsmall['Skill']))

Let's make a loop

In [32]:
count = 0
for i in range(0,len(CodeList)):
    name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
    try:
        dsmall = pd.read_csv(DataDir + '/Skills/Skills_' + name +'.csv')
    except CParserError as exception: 
        count = count + 1
    else:
        dsmall = dsmall.sort_values(by=['Skill'])
        dSkill.loc[i,:] = [CodeList[i]] + list(dsmall['Importance'])

In [33]:
len(dSkill),count

(1621, 226)

This occurred because some occupation do not have such data. So it gives a weird HTML file instead.

In [34]:
dSkill.head()

,SOC code,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Installation,Instructing,...,Science,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
0,25-1041.00,75,78,66,66,72,0,3,0,88,...,56,50,56,85,47,53,13,66,19,75
1,27-4011.00,47,60,53,53,63,31,41,38,50,...,3,50,50,56,44,47,28,50,47,60
2,13-1021.00,53,69,56,53,72,0,0,0,38,...,16,50,53,72,50,41,19,53,16,53
3,25-9021.00,72,88,72,72,69,0,3,0,72,...,44,72,69,78,66,66,25,66,3,72
4,13-1074.00,35,60,44,50,50,0,3,0,41,...,6,41,50,63,28,22,0,53,19,44


In [35]:
#Sort and remove redundant rows
dSkill = dSkill.sort_values(by=['SOC code'])
dSkill = dSkill.drop_duplicates()

In [36]:
dSkill.to_csv(DataDir + 'dSkill.csv')

### Stage 2.4: Construct tables of outcomes (Job Zone)

Next, let's look at **Job Zone**.

In [9]:
dJobZone = pd.DataFrame(columns = ['SOC code','JobZoneTitle','JobZoneSVPRange'])

In [10]:
count = 0
for i in range(0,len(CodeList)):
    name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
    try:
        dsmall = pd.read_csv(DataDir + '/Job_Zone/Job_Zone_' + name +'.csv')
    except CParserError as exception: 
        count = count + 1
    else:
        dJobZone.loc[i,'SOC code'] = CodeList[i]
        dJobZone.loc[i,'JobZoneTitle'] = dsmall.loc[0, 'Description']
        dJobZone.loc[i,'JobZoneSVPRange'] = dsmall.loc[5, 'Description']
        

In [11]:
len(dJobZone), count

(1623, 224)

In [12]:
dJobZone[:5]

,SOC code,JobZoneTitle,JobZoneSVPRange
0,25-1041.00,Job Zone Five: Extensive Preparation Needed,(8.0 and above)
1,27-4011.00,Job Zone Three: Medium Preparation Needed,(6.0 to < 7.0)
2,13-1021.00,Job Zone Four: Considerable Preparation Needed,(7.0 to < 8.0)
3,25-9021.00,Job Zone Five: Extensive Preparation Needed,(8.0 and above)
4,13-1074.00,Job Zone Two: Some Preparation Needed,(4.0 to < 6.0)


In [13]:
set(dJobZone.JobZoneSVPRange)

{'(4.0 to < 6.0)',
 '(6.0 to < 7.0)',
 '(7.0 to < 8.0)',
 '(8.0 and above)',
 '(Below 4.0)'}

Let's encode this into number.

In [14]:
dJobZone['JobZone'] = 0
for i in dJobZone.index:
    if dJobZone.loc[i,'JobZoneSVPRange'] == '(Below 4.0)':
        dJobZone.loc[i,'JobZone'] = 1
    elif dJobZone.loc[i,'JobZoneSVPRange'] == '(4.0 to < 6.0)':
        dJobZone.loc[i,'JobZone'] = 2
    elif dJobZone.loc[i,'JobZoneSVPRange'] == '(6.0 to < 7.0)':
        dJobZone.loc[i,'JobZone'] = 3
    elif dJobZone.loc[i,'JobZoneSVPRange'] == '(7.0 to < 8.0)':
        dJobZone.loc[i,'JobZone'] = 4
    else:
        dJobZone.loc[i,'JobZone'] = 5

In [15]:
dJobZone = dJobZone[['SOC code','JobZone']]
dJobZone = dJobZone.drop_duplicates()

In [16]:
dJobZone = dJobZone.sort_values(by=['SOC code'])
dJobZone.to_csv(DataDir + 'dJobZone.csv')

### Stage 2.5: Construct tables of features (Knowledge)

Next, let's look at **Knowledge**. The length is somewhat fixed (33). There are some exceptions (32). There are two such cases. Replace the missing values with zero.

In [42]:
#Get the list of skills from the first file
i=0
name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
dsmall = pd.read_csv(DataDir + '/Knowledge/Knowledge_' + name +'.csv')
dsmall = dsmall.sort_values(by=['Knowledge'])

In [45]:
dKnowledge = pd.DataFrame(columns = ['SOC code']+list(dsmall['Knowledge']))

In [46]:
count = 0
l = [item for item in range(0,len(CodeList)) if item not in [311,1298]]
for i in l:
    name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
    try:
        dsmall = pd.read_csv(DataDir + '/Knowledge/Knowledge_' + name +'.csv')
    except CParserError as exception: 
        count = count + 1
    else:
        dsmall = dsmall.sort_values(by=['Knowledge'])
        dKnowledge.loc[i,:] = [CodeList[i]] + list(dsmall['Importance'])
for i in [311,1298]:
    name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
    dsmall = pd.read_csv(DataDir + '/Knowledge/Knowledge_' + name +'.csv')
    dKnowledge.loc[i,:] = 0
    dKnowledge.loc[i,'SOC code'] = CodeList[i]
    for j in range(0,len(dsmall)):
        dKnowledge.loc[i,dsmall.loc[j,'Knowledge']] = dsmall.loc[j,'Importance']

In [47]:
len(dKnowledge), count

(1621, 226)

In [48]:
dKnowledge.head()

,SOC code,Administration and Management,Biology,Building and Construction,Chemistry,Clerical,Communications and Media,Computers and Electronics,Customer and Personal Service,Design,...,Philosophy and Theology,Physics,Production and Processing,Psychology,Public Safety and Security,Sales and Marketing,Sociology and Anthropology,Telecommunications,Therapy and Counseling,Transportation
0,25-1041.00,52,82,17,41,32,48,66,55,26,...,16,21,15,44,39,37,26,24,21,33
1,27-4011.00,32,2,12,10,29,77,78,61,37,...,15,12,48,28,31,18,17,73,6,17
2,13-1021.00,64,25,24,14,42,34,65,73,28,...,5,14,56,30,52,67,18,40,11,73
3,25-9021.00,63,72,15,55,52,69,57,76,24,...,24,21,23,51,42,39,46,30,29,25
4,13-1074.00,46,16,6,12,37,33,38,50,6,...,5,5,56,22,44,10,15,37,22,34


In [49]:
#Sort and remove redundant rows
dKnowledge = dKnowledge.sort_values(by=['SOC code'])
dKnowledge = dKnowledge.drop_duplicates()

In [50]:
dKnowledge.to_csv(DataDir + 'dKnowledge.csv')

### Stage 2.6: Found that Tools and technology is too large to take into account.

Finally, let's look at **tools and technology**.

In [228]:
TechnologyList = []
count = 0
for i in range(0,len(CodeList)):
    name = CodeList[i][0:7] + "-" + CodeList[i][8:10]
    try:
        dsmall = pd.read_csv(DataDir + '/Tools_Technology/Tools_Technology_'+ name +'.csv')
    except CParserError as exception: 
        count = count + 1
    else:
        TechnologyList = list(set(TechnologyList + list(dsmall.Category)))
len(TechnologyList)

4279

There are too many technologies. Not quite useful in this case. Just focus on skill and knowledge.

### Stage 2.7: Check out wage and projected job openings from U.S. Bureau of Labor Statistics (BLS).

In [229]:
filename = "EmploymentProjections.csv"
dWageAndOpenings = pd.pandas.read_csv(DataDir+filename)

In [230]:
len(dWageAndOpenings)

819

It has a relatively small number of rows. Let's try another sources.

In [51]:
filename = "occupation.xlsx"
dOpenings = pd.pandas.read_excel(DataDir+filename,sheetname="Table 1.2")

In [52]:
len(dOpenings)

1095

In [53]:
#Want only code name and job openings due to growth and replacemets 2014-2024
dOpenings = dOpenings[[dOpenings.columns[1],dOpenings.columns[2],dOpenings.columns[9]]]

In [54]:
dOpenings[:10]

,Unnamed: 1,Unnamed: 2,Unnamed: 9
0,NaN,Occupation Type,"Job openings due to growth and replacements, 2..."
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,00-0000,Summary,46506.9
4,11-0000,Summary,2586.8
5,11-1000,Summary,760
6,11-1011,Line item,58.4
7,11-1021,Line item,688.8
8,11-1031,Line item,12.9
9,11-2000,Summary,210.7


In [55]:
#Take only line item
dOpenings = dOpenings[dOpenings['Unnamed: 2']=="Line item"]
dOpenings = dOpenings[[dOpenings.columns[0],dOpenings.columns[2]]]

In [56]:
dOpenings.columns = ['SOC code', 'JobOpenings']

In [57]:
len(dOpenings)

819

Essentially the same number after removing summary rows.

In [58]:
#Screen out non-numeric data, sort and remove redundant
dOpenings = dOpenings[dOpenings['JobOpenings'].map(lambda x : isinstance(x, numbers.Number))]
dOpenings = dOpenings.sort_values(by=['SOC code'])
dOpenings = dOpenings.drop_duplicates()

In [59]:
dOpenings.to_csv(DataDir + 'dJobOpenings.csv')

Let's do the wage from BLS.

In [18]:
filename = "national_M2015_dl.xlsx"
dWage_BLS = pd.pandas.read_excel(DataDir+filename,sheetname="national_dl")

In [19]:
#Want only code name and median annual wage 
dWage_BLS = dWage_BLS[['OCC_CODE','A_MEDIAN']]
dWage_BLS.columns = ['SOC code', 'Wage_BLS']

In [20]:
dWage_BLS[:5]

,SOC code,Wage_BLS
0,00-0000,36200
1,11-0000,98560
2,11-1000,100940
3,11-1010,175110
4,11-1011,175110


In [21]:
#Screen out non-numeric data, sort and remove redundant
dWage_BLS = dWage_BLS[dWage_BLS['Wage_BLS'].map(lambda x : type(x) == int)]
dWage_BLS = dWage_BLS.sort_values(by=['SOC code'])
dWage_BLS = dWage_BLS.drop_duplicates()

In [22]:
len(dWage_BLS)

1379

In [23]:
dWage_BLS.to_csv(DataDir + 'dWage_BLS.csv')

### Stage 2.8: Check out risk of automation (Frey & Osbourne) and level of competition (O*NET)

In [3]:
filename = "Level_of_Competition.xls"
dLevelOfCompetition = pd.pandas.read_excel(DataDir+filename,sheetname="Browse by O-NET Data")

In [4]:
len(dLevelOfCompetition)

963

We want only "Context" and "Code". Rename them as 'SOC code' and 'LevelOfCompetition' 

In [5]:
dLevelOfCompetition = dLevelOfCompetition[['Code','Context']]
dLevelOfCompetition.columns = ['SOC code', 'LevelOfCompetition']

In [6]:
dLevelOfCompetition[:5]

,SOC code,LevelOfCompetition
0,27-2032.00,96
1,27-3043.05,95
2,27-2042.02,93
3,27-4021.00,91
4,41-9021.00,90


In [7]:
dLevelOfCompetition = dLevelOfCompetition.sort_values(by=['SOC code'])
dLevelOfCompetition = dLevelOfCompetition.drop_duplicates()
dLevelOfCompetition.to_csv(DataDir + 'dLevelOfCompetition.csv')

Next, let's look at risk of automation.

In [8]:
filename = "RiskOfAutomation.xlsx"
dRisk = pd.read_excel(DataDir+filename)

In [9]:
len(dRisk)

702

We want only "Probability" and "SOC code"

In [10]:
dRiskOfAutomation = dRisk[['SOC code','Probability']]
dRiskOfAutomation.columns = ['SOC code', 'RiskOfAutomation']

In [11]:
dRiskOfAutomation[:5]

,SOC code,RiskOfAutomation
0,29-1125,0.0028
1,49-1011,0.0030
2,11-9161,0.0030
3,21-1023,0.0031
4,29-1181,0.0033


In [12]:
dRiskOfAutomation = dRiskOfAutomation.sort_values(by=['SOC code'])
dRiskOfAutomation = dRiskOfAutomation.drop_duplicates()
dRiskOfAutomation.to_csv(DataDir + 'dRiskOfAutomation.csv')

### Stage 2.9: Check Job satisfaction from GSS

In [24]:
filename = "JobSatisfactionGSS.xls"
dJobSatisfaction_GSS = pd.read_excel(DataDir+filename,sheetname = 'Data')

In [25]:
dJobSatisfaction_GSS[:5]

,Gss year for this respondent,Respondent id number,Rs census occupation code (2010),Job or housework,Job satisfaction in general,Job satisfaction in general.1,Occupation code based on o*net
0,2000,1,Broadcast and sound engineering technicians an...,Very satisfied,Not applicable,Not applicable,0
1,2000,2,Secretaries and administrative assistants,Mod. satisfied,Not applicable,Not applicable,0
2,2000,3,Not applicable,No answer,Not applicable,Not applicable,0
3,2000,4,Veterinarians,Very satisfied,Not applicable,Not applicable,0
4,2000,5,Air traffic controllers and airfield operation...,A little dissat,Not applicable,Not applicable,0


In [26]:
len(dJobSatisfaction_GSS), len(set(dJobSatisfaction_GSS['Rs census occupation code (2010)'])),\
 len(set(dJobSatisfaction_GSS['Job or housework'])),len(set(dJobSatisfaction_GSS['Job satisfaction in general'])),\
len(set(dJobSatisfaction_GSS['Job satisfaction in general.1'])), \
len(set(dJobSatisfaction_GSS['Occupation code based on o*net']))

(21485, 509, 8, 8, 11, 431)

In [27]:
#Encode this into numbers
dJobSatisfaction_GSS['value1'] = None
for i in dJobSatisfaction_GSS.index:
    if dJobSatisfaction_GSS.loc[i,'Job or housework'] == 'Very satisfied':
        dJobSatisfaction_GSS.loc[i,'value1'] = 1.0
    elif dJobSatisfaction_GSS.loc[i,'Job or housework'] == 'Mod. satisfied':
        dJobSatisfaction_GSS.loc[i,'value1'] = 2.0/3
    elif dJobSatisfaction_GSS.loc[i,'Job or housework'] == 'A little dissat':
        dJobSatisfaction_GSS.loc[i,'value1'] = 1.0/3
    elif dJobSatisfaction_GSS.loc[i,'Job or housework'] == 'Very dissatisfied':
        dJobSatisfaction_GSS.loc[i,'value1'] = 0.0
    else:
        dJobSatisfaction_GSS.loc[i,'value1'] = None

In [28]:
#Encode this into numbers
dJobSatisfaction_GSS['value2'] = None
for i in dJobSatisfaction_GSS.index:
    if dJobSatisfaction_GSS.loc[i,'Job satisfaction in general'] == 'Very satisfied':
        dJobSatisfaction_GSS.loc[i,'value2'] = 1.0
    elif dJobSatisfaction_GSS.loc[i,'Job satisfaction in general'] == 'Somewhat satisfied':
        dJobSatisfaction_GSS.loc[i,'value2'] = 2.0/3
    elif dJobSatisfaction_GSS.loc[i,'Job satisfaction in general'] == 'Not too satisfied':
        dJobSatisfaction_GSS.loc[i,'value2'] = 1.0/3
    elif dJobSatisfaction_GSS.loc[i,'Job satisfaction in general'] == 'Not at all satisfied':
        dJobSatisfaction_GSS.loc[i,'value2'] = 0.0
    else:
        dJobSatisfaction_GSS.loc[i,'value2'] = None

In [29]:
#Encode this into numbers
dJobSatisfaction_GSS['value3'] = None
for i in dJobSatisfaction_GSS.index:
    if dJobSatisfaction_GSS.loc[i,'Job satisfaction in general.1'] == 'Completely satisfied':
        dJobSatisfaction_GSS.loc[i,'value3'] = 1.0
    elif dJobSatisfaction_GSS.loc[i,'Job satisfaction in general.1'] == 'Very satisfied':
        dJobSatisfaction_GSS.loc[i,'value3'] = 5.0/6
    elif dJobSatisfaction_GSS.loc[i,'Job satisfaction in general.1'] == 'Fairly satisfied':
        dJobSatisfaction_GSS.loc[i,'value3'] = 4.0/6
    elif dJobSatisfaction_GSS.loc[i,'Job satisfaction in general.1'] == 'Neither satisfied nor dissatisfied':
        dJobSatisfaction_GSS.loc[i,'value3'] = 3.0/6
    elif dJobSatisfaction_GSS.loc[i,'Job satisfaction in general.1'] == 'Fairly dissatisfied':
        dJobSatisfaction_GSS.loc[i,'value3'] = 2.0/6
    elif dJobSatisfaction_GSS.loc[i,'Job satisfaction in general.1'] == 'Very dissatisfied':
        dJobSatisfaction_GSS.loc[i,'value3'] = 1.0/6
    elif dJobSatisfaction_GSS.loc[i,'Job satisfaction in general.1'] == 'Completely dissatisfied':
        dJobSatisfaction_GSS.loc[i,'value3'] = 0.0
    else:
        dJobSatisfaction_GSS.loc[i,'value3'] = None

In [30]:
dJobSatisfaction_GSS['value'] = None
for i in dJobSatisfaction_GSS.index:
    dJobSatisfaction_GSS.loc[i,'value'] = 1.0*np.sum(dJobSatisfaction_GSS[['value1','value2','value3']].loc[i,:]) \
             /(3-np.sum(dJobSatisfaction_GSS[['value1','value2','value3']].loc[i,:].isnull()))

In [31]:
dJobSatisfaction_GSS[['value1','value2','value3','value']][:7]

,value1,value2,value3,value
0,1,None,None,1
1,0.666667,None,None,0.666667
2,None,None,None,NaN
3,1,None,None,1
4,0.333333,None,None,0.333333
5,0.666667,None,None,0.666667
6,1,None,None,1


In [32]:
#select only relevant columns
dJobSatisfaction_GSS = dJobSatisfaction_GSS[['Rs census occupation code (2010)','Occupation code based on o*net','value']]
dJobSatisfaction_GSS = dJobSatisfaction_GSS[dJobSatisfaction_GSS['value'].notnull()]

In [33]:
len(dJobSatisfaction_GSS)

14177

First, let's do matching between census occupation code and O*NET code. Fortunately, there is a table for conversion.

In [34]:
filename = "2010_OccCodeswithCrosswalkfrom2002-2011nov04.xls"
dCodeConversion = pd.read_excel(DataDir+filename,sheetname = '2010OccCodeList')

In [35]:
dCodeConversion = dCodeConversion.reset_index()
dCodeConversion = dCodeConversion[['US Census Bureau','Unnamed: 2']]
dCodeConversion.columns = ['Rs census occupation code (2010)','SOC code']
dCodeConversion = dCodeConversion.dropna()

In [36]:
dCodeConversion[:5]

,Rs census occupation code (2010),SOC code
3,Occupation 2010 Description,2010 SOC Code
9,"Management, Business, and Financial Occupations:",11-0000 - 13-0000
11,Management Occupations:,11-0000
13,Chief executives,11-1011
14,General and operations managers,11-1021


In [37]:
len(dCodeConversion)

568

In [38]:
dJobSatisfaction_GSS = dJobSatisfaction_GSS.merge(dCodeConversion, how ='left')

In [39]:
dJobSatisfaction_GSS[:5]

,Rs census occupation code (2010),Occupation code based on o*net,value,SOC code
0,Broadcast and sound engineering technicians an...,0,1,NaN
1,Secretaries and administrative assistants,0,0.666667,43-6010
2,Veterinarians,0,1,29-1131
3,Air traffic controllers and airfield operation...,0,0.333333,53-2020
4,Receptionists and information clerks,0,0.666667,43-4171


It seems that the occupation code from GSS is outdated. Use the new one. 

In [40]:
dJobSatisfaction_GSS = dJobSatisfaction_GSS[['SOC code','value']]
dJobSatisfaction_GSS = dJobSatisfaction_GSS.dropna()

In [41]:
len(dJobSatisfaction_GSS)

13323

In [42]:
dJobSatisfaction_GSS.columns = ['SOC code','JobSatisfaction_GSS']
dJobSatisfaction_GSS = dJobSatisfaction_GSS.sort_values(by=['SOC code'])
#YOu don't drop duplicates here since each row represents an individual
dJobSatisfaction_GSS.to_csv(DataDir + 'dJobSatisfaction_GSS.csv')

### Stage 2.10: Get data from PayScale - Job Satisfaction, Job meaning and wage

In [5]:
filename = "PayScale.csv"
dPayScale = pd.read_csv(DataDir+filename)

In [6]:
dPayScale[:5]

,Job Title,Median Pay,High Meaning,High Satisfaction
0,Clergy,"$46,600",98%,90%
1,"English Language and Literature Teachers, Post...","$43,600",96%,74%
2,Surgeons,"$304,000",96%,83%
3,"Directors, Religious Activities and Education","$37,600",96%,84%
4,"Education Administrators, Elementary and Secon...","$76,700",95%,88%


In [7]:
dPayScale['Median Pay'] = dPayScale['Median Pay'].map(lambda x: float(x[1:].replace(',', '')))
dPayScale['High Meaning'] = dPayScale['High Meaning'].map(lambda x: float(x.strip('%'))/100)

It turned out that there is something called 'Low Data' in our data. Remove it.

In [8]:
dPayScale = dPayScale[dPayScale['High Satisfaction']!='Low Data']

In [9]:
dPayScale['High Satisfaction'] = dPayScale['High Satisfaction']\
                                        .map(lambda x: float(x.strip('%'))/100)

It turned out that the job titles in PayScale match ones in O*NET database.

In [10]:
#First read the list of occupation 
filename = "All_Career_Clusters.csv"
df = pd.pandas.read_csv(DataDir+filename)

In [11]:
dPayScale = dPayScale.merge(df, right_on='Occupation',  left_on='Job Title', how ='left')

In [12]:
dPayScale = dPayScale[['Code', 'Median Pay', 'High Meaning', 'High Satisfaction']]
dPayScale = dPayScale.dropna()
dPayScale = dPayScale.drop_duplicates()

In [13]:
len(dPayScale)

435

In [14]:
dPayScale.columns = ['SOC code','Median Pay', 'High Meaning', 'High Satisfaction']
dPayScale = dPayScale.sort_values(by=['SOC code'])
dPayScale = dPayScale.drop_duplicates()
dPayScale.to_csv(DataDir + 'dPayScale.csv')

This concludes the data acquisation.